In [ ]:
import numpy as np
import pandas as pd
import warnings
import gc
import tensorflow as tf
from tensorflow import keras

import random
from random import choice
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Concatenate, Embedding, Flatten, Activation, Dropout
from sklearn.model_selection import KFold
from tensorflow.python.client import device_lib
warnings.filterwarnings('ignore')
import random

In [ ]:
import gc

## Data Preprocessing

In [ ]:
# load the data
import pandas as pd
dataframe = pd.read_csv("/content/drive/MyDrive/dlresearch/Practice_Log_Demographics.csv") 
dataframe.head()

,Unnamed: 0,term,Final_Exam,Final_Exam_Std,Final_Exam_Percent,Final_Exam_0_1,Final_Exam_log_trans,Dropped,SNPSHT_RPT_DT,STDNT_SEX_CD,gender,birthYear,birthMonth,STDNT_ASIAN_IND,STDNT_BLACK_IND,STDNT_HWIAN_IND,STDNT_HSPNC_IND,STDNT_NTV_AMRCN_IND,STDNT_WHITE_IND,STDNT_ETHNC_GRP_CD,ethnicity,STDNT_MULTI_ETHNC_IND,STDNT_HSPNC_LATINO_IND,nativeEnglish,FIRST_US_PRMNNT_RES_PSTL_CD,FIRST_US_PRMNNT_RES_PSTL_5_CD,FRST_FRGN_PRMNNT_RES_CNTRY_CD,permanentCountry,STDNT_CTZN_STAT_CD,USCitizenship,STDNT_CTZN_CNTRY_1_CD,citizenship,STDNT_CTZN_CNTRY_2_CD,STDNT_CTZN_CNTRY_2_DES,international,FIRST_TERM_ATTND_CD,firstTerm,FIRST_TERM_ATTND_BEGIN_YR_MO,FIRST_TERM_ATTND_END_YR_MO,LAST_TERM_ATTND_CD,...,Freshman,Junior,Sophomore,Senior,Minors_1,Minors_2OrMore,athlete_1,honorsPro_1,program_Cat,LSA,programBusiness,programEngineering,programInformation,programOther,interaction_count,activeCode_count,codelens_count,mChoice_count,pageVideoViews_count,parsons_count,interaction_days,spacing,user_id.x,practice_count,practice_days,user_id.y,course_name.y,chapter_label,sub_chapter_label,question_name,i_interval,e_factor,q,trials_num,day's_available_flashcards,start_practice,end_practice,timezoneoffset,next_eligible_date,days_offset
0,1,WN 2018,0.892,1.199159,89.2,0.883168,2.4681,0,09-FEB-21,1,Female,1998,5,0,0,0,0,0,1,1,White,0,0,Native,02052-3110,2052.0,NaN,NaN,1,U.S. Citzn,USA,United States,,,0,2110,FA 2016,2016/09,2016/12,2270,...,0,1,0,0,0,0,0,0,LS&A,1,0,0,0,0,5913.0,2965.0,556.0,636.0,1342.0,60.0,68.0,23.016143,148,493,44,148,UMSI106,Functions,DecodingaFunction,test_questionfunctions_3_3,1,1.96,1,6,14,2018-02-10 12:03:08,2018-02-10 12:04:01,0,0,23
1,2,WN 2018,0.892,1.199159,89.2,0.883168,2.4681,0,09-FEB-21,1,Female,1998,5,0,0,0,0,0,1,1,White,0,0,Native,02052-3110,2052.0,NaN,NaN,1,U.S. Citzn,USA,United States,,,0,2110,FA 2016,2016/09,2016/12,2270,...,0,1,0,0,0,0,0,0,LS&A,1,0,0,0,0,5913.0,2965.0,556.0,636.0,1342.0,60.0,68.0,23.016143,148,493,44,148,UMSI106,Tuples,UnpackingDictionaryItems,ee_ch09_05,1,1.40,0,0,8,2018-02-20 13:31:37,2018-02-20 13:31:41,0,0,33
2,3,WN 2018,0.892,1.199159,89.2,0.883168,2.4681,0,09-FEB-21,1,Female,1998,5,0,0,0,0,0,1,1,White,0,0,Native,02052-3110,2052.0,NaN,NaN,1,U.S. Citzn,USA,United States,,,0,2110,FA 2016,2016/09,2016/12,2270,...,0,1,0,0,0,0,0,0,LS&A,1,0,0,0,0,5913.0,2965.0,556.0,636.0,1342.0,60.0,68.0,23.016143,148,493,44,148,UMSI106,NestedData,DebuggingNestedData,ee_nested_data_011,15,2.50,4,2,3,2018-03-16 14:09:57,2018-03-16 14:10:33,0,0,57
3,4,WN 2018,0.892,1.199159,89.2,0.883168,2.4681,0,09-FEB-21,1,Female,1998,5,0,0,0,0,0,1,1,White,0,0,Native,02052-3110,2052.0,NaN,NaN,1,U.S. Citzn,USA,United States,,,0,2110,FA 2016,2016/09,2016/12,2270,...,0,1,0,0,0,0,0,0,LS&A,1,0,0,0,0,5913.0,2965.0,556.0,636.0,1342.0,60.0,68.0,23.016143,148,493,44,148,UMSI106,SimplePythonData,FunctionCalls,exercise_functionCalls_1,16,2.60,0,0,10,2018-01-30 14:15:49,2018-01-30 14:15:58,0,0,12
4,5,WN 2018,0.892,1.199159,89.2,0.883168,2.4681,0,09-FEB-21,1,Female,1998,5,0,0,0,0,0,1,1,White,0,0,Native,02052-3110,2052.0,NaN,NaN,1,U.S. Citzn,USA,United States,,,0,2110,FA 2016,2016/09,2016/12,2270,...,0,1,0,0,0,0,0,0,LS&A,1,0,0,0,0,5913.0,2965.0,556.0,636.0,1342.0,60.0,68.0,23.016143,148,493,44,148,UMSI106,SimplePythonData,DataTypes,test_question2_1_2,59,2.10,4,2,12,2018-03-13 14:22:54,2018-03-13 14:23:16,0,0,54


In [ ]:
#sort data based on timestamp
dataframe = dataframe.sort_values(by=['start_practice'])

In [ ]:
FEATURES = ['Repeated', 'chapter_label', 'sub_chapter_label','question_name','user_id','term',
            'STDNT_SEX_CD', 
            'NonNativeEnglish',
            'White',
            'Asian',
            'WhiteOrAsian',
            'Hispanic',
            'AfricanAmerican',
            'OtherEthnicities',
            'NonWhiteOrAsian',
            'STDNT_CTZN_STAT_CD', 'international', 
            'gradingType',
            'birthYear',
            'exclClassCumGPA',
            'Freshman',
            'Junior',
            'Sophomore',
            'Senior',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'athlete_1',
            'honorsPro',
            'LSA', 'programBusiness', 'programEngineering', 
            'programInformation', 'programOther',
            'HSCalculusTaken', 
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'classGraded', 'classHonors', 
            'Pass_Fail', 
            'parentsGraduateEdu',  'minorityGroup', 
            'q',
            'available_flashcards', 
            'start_practice', 
            'end_practice',
            'days_offset']

In [ ]:
dataframe['available_flashcards'] = dataframe["day's_available_flashcards"][:]
dataframe = dataframe.drop(["day's_available_flashcards"], axis=1)
dataframe['user_id'] = dataframe["user_id.y"][:]
dataframe = dataframe.drop(["user_id.y"], axis=1)
dataframe = dataframe.drop(["user_id.x"], axis=1)

In [ ]:
gc.collect()

61

In [ ]:
dataframe = dataframe[FEATURES]
gc.collect()

52

In [ ]:
dataframe = dataframe.fillna(0)
#transform q value so that if it is >= 4 it would be
#considered as 1 (correct) and if <4 0(incorrect)
dataframe['answer_correct'] = np.where(dataframe['q']>=4, 1, 0)
dataframe['answer_correct'].mean()

0.5326395552433993

In [ ]:
#label encode term, chapter_label, question_name, sub_chapter_label columns
dataframe['term'] = dataframe['term'].astype('category')
dataframe['user_id'] = dataframe['user_id'].astype(int)
dataframe['user_id'] = dataframe['user_id'].astype(str)
dataframe['user_id'] = dataframe['term'].str.cat(dataframe['user_id'], sep=':')
dataframe['user_id'] = dataframe['user_id'].astype('category')
dataframe['chapter_label'] = dataframe['chapter_label'].astype('category')
dataframe['sub_chapter_label'] = dataframe['sub_chapter_label'].astype('category')
dataframe['question_name'] = dataframe['question_name'].astype('category')


In [ ]:
#calculate time_lag and prev_time_elapsed
dataframe['prev_time_elapsed'] = None
dataframe['time_lag'] = None
dataframe['time_lag'] = dataframe['time_lag'].astype(np.float)
dataframe['prev_time_elapsed'] = dataframe['prev_time_elapsed'].astype(np.float)
dataframe.start_practice = pd.to_datetime(dataframe.start_practice, format='%Y-%m-%d %H:%M:%S')
dataframe.end_practice = pd.to_datetime(dataframe.end_practice, format='%Y-%m-%d %H:%M:%S')

In [ ]:
from tqdm import tqdm
for user in tqdm(dataframe['user_id'].unique()):
    tmp_user = dataframe[dataframe['user_id']==user]
    tmp_time_elapsed = tmp_user.end_practice.apply(lambda a: a.timestamp()) - tmp_user.start_practice.apply(lambda a: a.timestamp())
    #shifting time elapsed by one
    #so that time_elapsed row for each question
    #would refer to the time that user took to answer
    #previous question
    tmp_time_elapsed = np.insert(np.array(tmp_time_elapsed[:-1]), 0, -1., axis=0)
    indices = tmp_user.index
    start_row = indices[0]
    dataframe['time_lag'].iloc[start_row] = -1
    time_substrahend = tmp_user.start_practice.iloc[:-1]
    time_substrahend = time_substrahend.apply(lambda a: a.timestamp())
    time_substrahend = np.array(time_substrahend)

    time_minuend = tmp_user.start_practice.iloc[1:]
    time_minuend = time_minuend.apply(lambda a: a.timestamp())
    time_minuend = np.array(time_minuend)
    
    dataframe['prev_time_elapsed'].iloc[indices] = tmp_time_elapsed
    dataframe['time_lag'].iloc[indices[1:]] = time_minuend - time_substrahend

100%|██████████| 887/887 [00:11<00:00, 79.54it/s]


In [ ]:
# remove students who are taking the course for a second time
dataframe  = dataframe[dataframe['Repeated'] == ' ']
gc.collect()

50

In [ ]:
#drop column end_practice
dataframe.drop(columns=['end_practice'], inplace=True)

In [ ]:
# calculate the age feature
dataframe['term_value'] = [int(ele[3:]) for ele in dataframe['term']]
dataframe['age'] = dataframe['term_value'] - dataframe['birthYear']

In [ ]:
# drop term_value and birthYear column
dataframe.drop(columns=['term_value', 'birthYear'], inplace=True)

In [ ]:
# convert minors_count to int value
new_minors_count = []
for i in dataframe['minorsCount']:
  if i == 0 or i == '0':
    new_minors_count.append(0)
  elif i == '1 Minor':
    new_minors_count.append(1)
  else:
    new_minors_count.append(2)

dataframe['minorsCount'] = new_minors_count

In [ ]:
print("we have ", dataframe['user_id'].nunique()," users in total.")

we have  881  users in total.


In [ ]:
for category in ['term','chapter_label', 'sub_chapter_label', 'question_name']:
  dataframe[category] =  dataframe[category].cat.codes


In [ ]:
NUMERIC_FEATURE =  ['age',
            'exclClassCumGPA',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'available_flashcards', 
            'days_offset', 
            'prev_time_elapsed',
             'time_lag']
# z-score normalize the numerical features
for f in NUMERIC_FEATURE:
  m = dataframe[f].mean()
  std = dataframe[f].std()
  dataframe[f] = (dataframe[f] - m)/std

In [ ]:
FEATURE_TRANS =  ['answer_correct', 'chapter_label', 'sub_chapter_label','question_name','user_id','term',
                  'STDNT_SEX_CD', 
                    'White','Asian','NonWhiteOrAsian',
            'STDNT_CTZN_STAT_CD', 'international', 
            'age',
            'exclClassCumGPA',
           'Freshman',
            'Junior',
            'Sophomore',
            'Senior',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'athlete_1',
            'honorsPro',
            'LSA', 'programBusiness', 'programEngineering', 
            'programInformation', 'programOther',
            'HSCalculusTaken', 
            'highSchoolGPA', 

            'majorsCount', 'minorsCount',

            'PREV_TERM_CUM_GPA',
            'classGraded', 'classHonors', 
            'Pass_Fail', 
            'parentsGraduateEdu',  'minorityGroup', 
            'available_flashcards', 
            'days_offset', 'prev_time_elapsed',
             'time_lag']
grouped_data = dataframe[FEATURE_TRANS].groupby(['user_id']).apply(lambda r: (
                r['answer_correct'],
                r['term'],
                r['chapter_label'],
                r['sub_chapter_label'],
                r['question_name'],
                np.array([r['STDNT_SEX_CD'],r['STDNT_CTZN_STAT_CD'], r['international'], 
                  r['White'],r['Asian'],r['NonWhiteOrAsian'],
                 r['age'],r['exclClassCumGPA'],
                r['Freshman'], r['Junior'], r['Sophomore'], r['Senior'],
                r['termCreditsGPA'], r['termCreditsNoGPA'],
                r['athlete_1'], r['honorsPro'],
                r['LSA'], r['programBusiness'], r['programEngineering'], 
                r['programInformation'], r['programOther'],
                r['HSCalculusTaken'],  r['highSchoolGPA'], 
                r['majorsCount'], r['minorsCount'],
                r['PREV_TERM_CUM_GPA'], 
                r['parentsGraduateEdu'], r['minorityGroup'],
                r['available_flashcards'],
                r['days_offset'],
                r['prev_time_elapsed'],
                r['time_lag']
              ]).transpose()
                ))

In [ ]:
gc.collect()

50

In [ ]:
# remove students who don't have make any interactions with the tool
toRemove = []
for index in grouped_data.index:
  if len(grouped_data[index][0]) <= 10:
    toRemove.append(index)
grouped_data = grouped_data.drop(index=toRemove)

In [ ]:
#SETTINGS -> can be modified at any time
MAXLENGTH = 500
EMBEDDING_DIM = 128
DENSE_NEURON = 16
LSTM_NEURON = 32

In [ ]:
FEATURES_SIZE = 38
CHAPTER_SIZE = 38
SUB_CHAPTER_SIZE = 222
QUESTION_SIZE = 1065

In [ ]:
#create dataset class
#to prepare it for train, valid, and test sets
from torch.utils.data import Dataset, DataLoader
class SPACE_DATASET(Dataset):
    def __init__(self, data, maxlength = 100):
        super(SPACE_DATASET, self).__init__()
        self.maxlength = maxlength
        self.data = data
        self.users = list()
        for user in data.index:
            self.users.append(user)

    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, ix):
        user = self.users[ix]
        user = user
        target, term, ch_label, sub_ch_label, ques_name, features = self.data[user]
        
        #0s should be used as padding values
        ori_target = target.values 
        term = term.values
        ch_label = ch_label.values + 1
        sub_ch_label = sub_ch_label.values +1
        ques_name = ques_name.values + 1
        
        n = len(ch_label)

        # one hot for term
        term_encode = [0]*6
        term_encode[term[0]] = 1
        shifted_target= []

        
        # get  user interaction informations in the previous MAXLEN interactions
        if n > self.maxlength:
          ch_label = ch_label[-self.maxlength:]
          sub_ch_label = sub_ch_label[-self.maxlength:]
          ques_name = ques_name[-self.maxlength:]
          features = features[-self.maxlength:]
          target = ori_target[-self.maxlength:]
          shifted_target = ori_target[ (-self.maxlength - 1) :-1]
        else:
          ch_label = [0]*(self.maxlength - n)+list(ch_label[:])
          sub_ch_label = [0]*(self.maxlength - n)+list(sub_ch_label[:])
          ques_name = [0]*(self.maxlength - n)+list(ques_name[:])
          features = [[0]*len(features[0])]*(self.maxlength  - n)+list(features[:])
          target = [-1]*(self.maxlength - n) + list(ori_target[:])
          shifted_target = [-1]*(self.maxlength + 1 - n) + list(ori_target[:-1])

        new_features = []
        count = 0
        for f in features:
          temp = list(f)
          temp.extend(term_encode)
          # temp.append(shifted_target[count]) #uncomment this line for include previous response feature
          new_features.append(temp)
          
          count += 1
        
        features = new_features
        
        return ch_label,sub_ch_label,ques_name,features,target

## KFOLD - LSTM


In [ ]:
# 5 fold cross validation with GRU-based model
import torch
X = np.array(grouped_data.keys())
kfold = KFold(n_splits=5, shuffle=True)
train_losses = list()
train_aucs = list()
val_losses = list()
val_aucs = list()
train_eval= list()
test_eval = list()
for train, test in kfold.split(X):
    users_train, users_test =  X[train], X[test]
    n = len(users_test)//2
    users_test, users_val = users_test[:n], users_test[n: ]
    train_data_space = SPACE_DATASET(grouped_data[users_train], MAXLENGTH)
    val_data_space = SPACE_DATASET(grouped_data[users_val], MAXLENGTH)
    test_data_space = SPACE_DATASET(grouped_data[users_test], MAXLENGTH)
    #construct training input
    train_chapter=[]
    train_sub_chapter=[]
    train_question = []
    train_features=[]
    train_labels=[]
    for i in range(len(users_train)):
        user = train_data_space.__getitem__(i)
        train_chapter.append(user[0])
        train_sub_chapter.append(user[1]) 
        train_question.append(user[2])
        train_features.append(user[3])
        train_labels.append(user[4])
    train_chapter = np.array(train_chapter)
    train_sub_chapter = np.array(train_sub_chapter)
    train_question = np.array(train_question)
    train_features = np.array(train_features)
    train_labels= np.array(train_labels)[..., np.newaxis]

    #construct validation input
    val_chapter=[]
    val_sub_chapter=[]
    val_question = []
    val_features=[]
    val_labels=[]
    for i in range(len(users_val)):
        user = val_data_space.__getitem__(i)
        val_chapter.append(user[0])
        val_sub_chapter.append(user[1]) 
        val_question.append(user[2])
        val_features.append(user[3])
        val_labels.append(user[4])
    val_chapter = np.array(val_chapter)
    val_sub_chapter = np.array(val_sub_chapter)
    val_features = np.array(val_features)
    val_question = np.array(val_question)
    val_labels= np.array(val_labels)[..., np.newaxis]

    # construct test input
    test_chapter=[]
    test_sub_chapter=[]
    test_features=[]
    test_question=[]
    test_labels=[]
    for i in range(len(users_test)):
        user = test_data_space.__getitem__(i)
        test_chapter.append(user[0])
        test_sub_chapter.append(user[1]) 
        test_question.append(user[2])
        test_features.append(user[3])
        test_labels.append(user[4])
    test_chapter = np.array(test_chapter)
    test_sub_chapter = np.array(test_sub_chapter)
    test_features = np.array(test_features)
    test_question = np.array(test_question)
    test_labels= np.array(test_labels)[..., np.newaxis]

    # define loss function and evaluation metrics
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    acc = tf.keras.metrics.Accuracy()
    auc = tf.keras.metrics.AUC()

    def masked_bce(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return bce(flat_ground_truth, flat_pred, sample_weight=label_mask)

    def masked_acc(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      flat_pred = (flat_pred >= 0.5)
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return acc(flat_ground_truth, flat_pred, sample_weight=label_mask)

    def masked_auc(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return auc(flat_ground_truth, flat_pred, sample_weight=label_mask)

    # input layer
    input_chap = tf.keras.Input(shape=(MAXLENGTH))
    input_sub_chap = tf.keras.Input(shape=(MAXLENGTH))
    input_ques =  tf.keras.Input(shape=(MAXLENGTH))
    input_features = tf.keras.Input(shape=(MAXLENGTH, FEATURES_SIZE))

    # embedding layer for categorical features
    embedding_chap = Embedding(input_dim = CHAPTER_SIZE, output_dim = EMBEDDING_DIM)(input_chap)
    embedding_sub_chap = Embedding(input_dim = SUB_CHAPTER_SIZE, output_dim = EMBEDDING_DIM)(input_sub_chap) 
    embedding_ques = Embedding(input_dim = QUESTION_SIZE, output_dim = EMBEDDING_DIM)(input_ques)       

    # dense layer for numeric features
    dense_features = Dense(EMBEDDING_DIM,input_shape = (None, MAXLENGTH))(input_features)



    # lstm layers
    lstm_chap = LSTM(LSTM_NEURON, input_shape = (None, EMBEDDING_DIM),return_sequences = True)(embedding_chap)
    lstm_sub_chap = LSTM(LSTM_NEURON, input_shape = (None, EMBEDDING_DIM),return_sequences = True)(embedding_sub_chap)
    lstm_ques = LSTM(LSTM_NEURON, input_shape = (None, EMBEDDING_DIM),return_sequences = True)(embedding_ques)
    lstm_features = LSTM(LSTM_NEURON, input_shape = (None, EMBEDDING_DIM),return_sequences = True)(dense_features)

    lstm_output = tf.concat([lstm_chap, lstm_sub_chap, lstm_ques,lstm_features], axis = 2)

    dense1 = Dense(256, input_shape = (None, 4*EMBEDDING_DIM), activation='relu')(lstm_output)
    dropout1 = Dropout(0.1)(dense1)
    dense2 = Dense(64, input_shape = (None, 256), activation='relu')(dropout1)
    dropout2 = Dropout(0.1)(dense2)
    pred = Dense(1, input_shape = (None, 64), activation='sigmoid')(dropout2)

    model = tf.keras.Model(
        inputs=[input_chap, input_sub_chap,input_ques, input_features],
        outputs=pred,
        name='lstm_model'
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt_adam = Adam(learning_rate = 0.005)
    model.compile(
        optimizer=opt_adam,
        loss= masked_bce,
        metrics = [masked_acc, masked_auc]
    )

    history = model.fit(
      [train_chapter, train_sub_chapter, train_question, train_features],
      train_labels,
      batch_size = 64,
      epochs = 100,
      validation_data=([val_chapter, val_sub_chapter, val_question, val_features], val_labels),
      callbacks=[callback]
    )
    val_losses.append(list(history.history['val_loss']))
    train_losses.append(list(history.history['loss']))
    val_aucs.append(list(history.history['val_masked_auc']))
    train_aucs.append(list(history.history['masked_auc']))
    train_score = model.evaluate([train_chapter, train_sub_chapter, train_question, train_features], train_labels)
    train_eval.append(train_score)
    test_score = model.evaluate([test_chapter, test_sub_chapter, test_question, test_features], test_labels)
    test_eval.append(test_score)
    print("Test: ", test_score)
    def reset_weights(model):
      for layer in model.layers: 
        if isinstance(layer, tf.keras.Model):
          reset_weights(layer)
          continue
        for k, initializer in layer.__dict__.items():
          if "initializer" not in k:
            continue
          # find the corresponding variable
          var = getattr(layer, k.replace("_initializer", ""))
          var.assign(initializer(var.shape, var.dtype))
    reset_weights(model)

Epoch 1/100
11/11 [==============================] - 40s 363ms/step - loss: 0.6194 - masked_acc: 0.5216 - masked_auc: 0.5270 - val_loss: 0.5419 - val_masked_acc: 0.6136 - val_masked_auc: 0.6551
Epoch 2/100
11/11 [==============================] - 1s 114ms/step - loss: 0.5430 - masked_acc: 0.6313 - masked_auc: 0.6790 - val_loss: 0.4907 - val_masked_acc: 0.6609 - val_masked_auc: 0.7197
Epoch 3/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4921 - masked_acc: 0.6705 - masked_auc: 0.7320 - val_loss: 0.4761 - val_masked_acc: 0.6863 - val_masked_auc: 0.7520
Epoch 4/100
11/11 [==============================] - 1s 103ms/step - loss: 0.4774 - masked_acc: 0.6916 - masked_auc: 0.7586 - val_loss: 0.4674 - val_masked_acc: 0.7013 - val_masked_auc: 0.7699
Epoch 5/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4689 - masked_acc: 0.7047 - masked_auc: 0.7738 - val_loss: 0.4671 - val_masked_acc: 0.7112 - val_masked_auc: 0.7813
Epoch 6/100
11/11 [===============

In [ ]:
t_eval = np.array(test_eval)
print("test avg loss: ", np.mean(t_eval[:, 0]), "+/-" ,np.std(t_eval[:, 0]))
print("test avg acc: ", np.mean(t_eval[:, 1]),  "+/-" ,np.std(t_eval[:, 1]))
print("test avg auc: ", np.mean(t_eval[:, 2]), "+/-" ,np.std(t_eval[:, 2]))

test avg loss:  0.4956098973751068 +/- 0.009004329765390178
test avg acc:  0.7524508953094482 +/- 0.004915822987043772
test avg auc:  0.8266466975212097 +/- 0.005394693000566721


In [ ]:
t_eval = np.array(train_eval)
print("train avg loss: ", np.mean(t_eval[:, 0]), "+/-" ,np.std(t_eval[:, 0]))
print("train avg acc: ", np.mean(t_eval[:, 1]),  "+/-" ,np.std(t_eval[:, 1]))
print("train avg auc: ", np.mean(t_eval[:, 2]), "+/-" ,np.std(t_eval[:, 2]))

train avg loss:  0.4279987752437592 +/- 0.011397843749356646
train avg acc:  0.7519938945770264 +/- 0.004975242841066693
train avg auc:  0.8261496067047119 +/- 0.005418478650433621
